In [9]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats
import os

In [1]:
output_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files"

Buildings processing
- create raster topo from contours
- subtract out contours from building heights to extrude in rhino

In [2]:
default_height = 20

In [5]:
extent_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/neighborhood_extent.shp"
extent = gp.read_file(extent_path)

In [6]:
# set watershed boundary
watershed_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/green_duwamish_watershed.shp"
watershed = gp.read_file(watershed_path)

In [7]:
# find intersection of watershed boundary and extent
watershed_extent_intersect = gp.overlay(watershed, extent, how='intersection')

In [16]:
watershed_extent_intersect.to_file(os.path.join(output_path, "watershed_boundary_intersect.shp"))

In [24]:
# roads processing
roads_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
roads = gp.read_file(roads_path)
roads_clip = gp.overlay(roads, watershed_extent_intersect, how='intersection')
roads_clip.to_file(os.path.join(output_path, "neighborhood_streets.shp"))

In [8]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [11]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)

In [12]:
buildings_clip = gp.overlay(buildings, watershed_extent_intersect, how = "intersection")

In [13]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')

In [14]:
buildings_merge.set_geometry('geometry_y', inplace = True)

In [28]:
# fill 0 and NA values to default_height
buildings_merge['BP99_APEX'].fillna(default_height, inplace = True)
buildings_merge['BP99_APEX'] = np.where(buildings_merge['BP99_APEX']==0,default_height, buildings_merge['BP99_APEX'])

In [30]:
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file(os.path.join(output_path,"neighborhood_buildings.shp"))

In [17]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/min_bath_topo.tif"

In [18]:
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")

/home/nweiss/miniconda3/envs/gis/lib/python3.11/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [19]:
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)

In [20]:
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')

In [21]:
merged_heights['height_upd'] = np.where((merged_heights['BP99_APEX'].isna()==False) | (merged_heights['mean'].isna()==False), default_height, (merged_heights['BP99_APEX']-merged_heights['mean']))

In [23]:
merged_heights['height_upd'].describe()

count    20069.0
mean        20.0
std          0.0
min         20.0
25%         20.0
50%         20.0
75%         20.0
max         20.0
Name: height_upd, dtype: float64

In [42]:
merged_heights.set_geometry('geometry_y')

,index,OBJECTID,BP99_TYPE,BP99_APEX,BP99_BLDGK,FEATURE_ID,YEARUPDATE,SHAPE_Leng,SHAPE_Area,geometry_x,...,Shape__Len,id,geometry_y,min,max,mean,count,std,median,height_upd
0,0,263015,BLD,81.0,1603149.0,96359.0,NaN,137.581704,954.986190,POINT (1269718.180 201502.410),...,305066.764574,NaN,"POLYGON Z ((1269728.474 201505.377 0.000, 1269...",-21.110571,-13.896954,-15.643722,38,1.962462,-14.625418,20.0
1,1,263016,BLD,66.0,1633032.0,332792.0,NaN,164.827831,1059.250511,POINT (1269613.743 201630.253),...,305066.764574,NaN,"POLYGON Z ((1269626.389 201657.008 0.000, 1269...",-19.431469,-15.849288,-17.753135,41,1.158415,-17.612436,20.0
2,2,263017,BLD,67.0,1633034.0,254322.0,NaN,173.643943,1212.567710,POINT (1269690.633 201612.189),...,305066.764574,NaN,"POLYGON Z ((1269706.538 201640.549 0.000, 1269...",-17.399797,-11.026821,-16.176445,51,1.340995,-16.620689,20.0
3,3,263018,BLD,64.0,1603150.0,96373.0,NaN,126.707617,799.976889,POINT (1269794.520 201491.913),...,305066.764574,NaN,"POLYGON Z ((1269806.989 201513.120 0.000, 1269...",-20.673250,-16.524836,-18.839761,33,1.224284,-19.055712,20.0
4,4,263019,BLD,333.0,1370925.0,174844.0,NaN,155.019904,969.706351,POINT (1277358.201 205642.742),...,305066.764574,NaN,"POLYGON Z ((1277365.344 205658.228 0.000, 1277...",318.770691,321.426575,319.810907,38,0.548536,319.763428,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18447,18447,526892,BLD,85.0,0.0,529445.0,2015,279.660361,4450.874551,POINT (1271445.263 192360.474),...,305066.764574,NaN,"POLYGON Z ((1271490.789 192384.533 0.000, 1271...",56.987404,59.241844,58.212663,180,0.612540,58.207108,20.0
18448,18448,526948,BLD,42.0,0.0,504530.0,2015,487.845238,13246.238895,POINT (1271745.605 198319.518),...,305066.764574,NaN,"POLYGON Z ((1271810.141 198267.562 0.000, 1271...",9.813066,11.467998,10.704821,525,0.408358,10.736494,20.0
18449,18449,526949,BLD,55.0,0.0,504530.0,2015,748.231186,25748.409061,POINT (1271678.997 198226.549),...,305066.764574,NaN,"POLYGON Z ((1271555.667 198178.787 0.000, 1271...",10.164929,12.602659,11.524145,1034,0.607931,11.654552,20.0
18450,18450,526950,BLD,38.0,0.0,504530.0,2015,360.422900,7628.829111,POINT (1271446.646 198324.104),...,305066.764574,NaN,"POLYGON Z ((1271491.239 198365.218 0.000, 1271...",11.180118,12.188333,11.694040,311,0.260908,11.704866,20.0


In [43]:
merged_heights[['OBJECTID','BP99_APEX','mean','min','max','median','std','height_upd','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/building_heights_upd.shp")

TOPO PROCESSING
- merging bathymetry data with topography dem

In [44]:
# mask topo_rast by extent

# convert to ascii file
